# 🏠 House Price Prediction with Multiple ML Models

This project predicts house prices using different machine learning models and compares their accuracy and training time.  
We'll use:
- **Python** for implementation
- **NumPy** & **Pandas** for data handling
- **Matplotlib** for visualization
- **TensorFlow** for neural networks

## Goals:
1. Load and explore the dataset
2. Preprocess data (handle missing values, scaling, encoding)
3. Train multiple models
4. Compare accuracy & execution time
5. Visualize results

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from tensorflow import keras


Matplotlib is building the font cache; this may take a moment.
